In [2]:
import os
import csv
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm  # For progress monitoring
import re
import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# # Install py7zr if it's not already installed
# !pip install py7zr

# import py7zr

# # Define file paths
# compressed_file_path = '/content/drive/MyDrive/Colab Notebooks/AIFFEL_DATATHONE(2조)/train.tsv.7z'
# extract_dir = '/content/drive/MyDrive/Colab Notebooks/AIFFEL_DATATHONE(2조)/'
# extracted_file_path = os.path.join(extract_dir, 'train.tsv')

# # Extract the .7z file
# with py7zr.SevenZipFile(compressed_file_path, mode='r') as z:
#     z.extractall(path=extract_dir)

# # Check if the file was extracted successfully
# if os.path.exists(extracted_file_path):
#     print(f"File extracted successfully to {extracted_file_path}")
# else:
#     print("Extraction failed or file not found")

File extracted successfully to /content/drive/MyDrive/Colab Notebooks/AIFFEL_DATATHONE(2조)/train.tsv


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the TSV file
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIFFEL_DATATHONE(2조)/train.tsv', sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Set the display option to show the entire column value
pd.set_option('display.max_colwidth', None)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  float64
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  float64
 3   category_name      1476208 non-null  object 
 4   brand_name         849853 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  float64
 7   item_description   1482529 non-null  object 
dtypes: float64(4), object(4)
memory usage: 90.5+ MB


In [ ]:
train_df.isnull().sum()

train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          6
dtype: int64

In [ ]:
train_df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,train_id,item_condition_id,price,shipping
count,1392726.00000,1392726.00000,1392726.00000,1392726.00000
mean,741315.39926,1.90187,27.04392,0.44993
std,427946.35441,0.90459,39.07712,0.49749
min,1.00000,1.00000,3.00000,0.00000
25%,370620.25000,1.00000,11.00000,0.00000
50%,741368.50000,2.00000,17.00000,0.00000
75%,1111926.75000,3.00000,30.00000,1.00000
max,1482534.00000,5.00000,2009.00000,1.00000


# 이상치 처리

In [ ]:
# 이상치 처리
train_df = train_df[train_df['price'] != 0]

# 이상치 처리 확인
train_df.sort_values(by='price', ascending=False)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
760469,760469.0,NEW Chanel WOC Caviar Gold Hardware,1.0,Women/Women's Handbags/Shoulder Bag,Chanel,2009.0,0.0,"New with tag, box and dustbag but no receipt. Dark purple color Cross body shoulder bag Gold hardware You must purchase and pay for two listing for the total [rm] or your purchase will be cancelled. Smoke free and Pet free home. PRICE IS FIRM thank you"
1262245,1262245.0,NEW-Chanel Boy Wallet o Chain WOC Caviar,1.0,Women/Women's Handbags/Messenger & Crossbody,Chanel,2006.0,0.0,Brand: Chanel Style: WOC Boy Wallet on Chain Type: Crossbody shoulder bag Color: Black with ruthenium hardware Material: Caviar leather Condition: New with original packaging& copy of receipt Total price is [rm]. Must purchase and pay for both listings to be valid.
1393600,1393600.0,David Yurman Wheaton ring,2.0,Women/Jewelry/Rings,David Yurman,2004.0,0.0,David Yurman Wheaton wing. Size 6. Original receipt included (may have to black out some information) as well as David Yurman pouch and polishing cloth. Like new condition. Worn twice.
415027,415027.0,Chanel Classic Jumbo Single flap bag,3.0,Women/Women's Handbags/Shoulder Bag,Chanel,2000.0,1.0,"Authentic. Pre-loved in Excellent Condition. Pen mark on the bottom side of the bag but won't be noticed when worn, shown in the 2nd picture. Must be bought with the other listing, [rm]. Total of [rm]."
778940,778940.0,Mary kay,1.0,Beauty/Makeup/Face,Mary Kay,2000.0,1.0,30 time wise sets Oily to combo skin
...,...,...,...,...,...,...,...,...
1320473,1320473.0,☆ TATTOO CHOKER ☆,1.0,Women/Jewelry/Necklaces,NaN,3.0,1.0,"LAST ONE LEFT !!!!! CUTE AND TRENDY PLASTIC TATTOO CHOKER ! ONLY [rm], VERY FEW IN STOCK! FREE SHIP FREE SHIPPING Shipping Details: ☆Tracking number is included and usually updates within 24hrs. SOMETIMES IT DOES NOT SCAN UNTIL IT REACHES YOUR CITY, PLEASE BE PATIENT. ☆Please allow 2-7 Business days. MAY TAKE LONGER DUE TO USPS. ☆I can bundle, comment on any of my posts. :) Happy Shopping!!! COMPARE TO: FOREVER 21, BRANDY MELVILLE, H&M, CLAIRES, ZARA, HOT TOPIC, CHARLOTTE RUSSE ♡SANDY LOVE SHOP♡"
1421106,1421106.0,3D ankle socks,1.0,Women/Athletic Apparel/Socks,NaN,3.0,1.0,"Dream catcher Listing for active users, super cute , for adult and teenagers"
1320462,1320462.0,Blackheads removers free shipping!!,1.0,Beauty/Skin Care/Face,NaN,3.0,1.0,2 blackheads removers
1468628,1468628.0,Hollywood Spaniel book,1.0,Other/Books/Children's Books,NaN,3.0,1.0,Hollywood Spaniel softcover new. FREE SHIPPING


# 결측치 처리

## 'category_name' 결측치 삭제

In [ ]:
train_df = train_df.dropna(subset=['category_name'])

## 'item_description' 유사 결측치 삭제

In [ ]:
# 유사 결측치 확인 및 삭제

# Identify rows where 'item_description' contains 'no description'
no_description_mask = train_df['item_description'].str.contains('no description', case=False, na=False)

# Replace 'item_description' in those rows with NaN
train_df.loc[no_description_mask, 'item_description'] = np.nan

# Drop rows where 'item_description' is NaN
train_df = train_df.dropna(subset=['item_description'])

## 'category_name' 자르기

In [ ]:
# Function to split the category_name column and ensure it always returns 3 elements
def split(category_name):
    try:
        parts = category_name.split('/')
        # Ensure the returned list is always of length 3
        return (parts + ['Null', 'Null', 'Null'])[:3]
    except AttributeError:
        return ['Null', 'Null', 'Null']

category_1 = []
category_2 = []
category_3 = []

for i in range(train_df.shape[0]):
    temp = split(train_df['category_name'].iloc[i])  # train_df['category_name'][i] 대신 train_df['category_name'].iloc[i] 사용
    category_1.append(temp[0])
    category_2.append(temp[1])
    category_3.append(temp[2])

train_df['category_1'] = category_1
train_df['category_2'] = category_2
train_df['category_3'] = category_3

print('1st Category:\n', train_df['category_1'].value_counts())
print('2nd Category:', train_df['category_2'].nunique())
print('3rd Category:', train_df['category_3'].nunique())

1st Category:
 category_1
Women                     626531
Beauty                    199877
Kids                      160906
Electronics               115109
Men                        87837
Home                       62225
Vintage & Collectibles     44108
Other                      42801
Handmade                   29769
Sports & Outdoors          24124
Name: count, dtype: int64
2nd Category: 113
3rd Category: 869


## 텍스트 컬럼 전처리

In [ ]:
# Text column preprocessing

text_cols = ['name', 'brand_name', 'item_description', 'category_1', 'category_2', 'category_3']

def preprocess_text(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove non-alphanumeric characters (except spaces and accented characters)
        text = re.sub(r'[^\w\s\u00C0-\u017F]', '', text)
        # Remove extra whitespace
        text = ' '.join(text.split())
        # Strip leading and trailing whitespace and convert empty strings to None (NaN in pandas)
        text = text.strip()
        if text == '':
            text = None
    return text

# Initialize tqdm
tqdm.pandas()

# Apply preprocessing with progress bar
for col in text_cols:
    print(f"Processing column: {col}")
    train_df[col] = train_df[col].progress_apply(preprocess_text)
    print(f"Finished processing {col}")

print("All text preprocessing completed!")

Processing column: name


100%|██████████| 1393287/1393287 [00:12<00:00, 115374.78it/s]


Finished processing name
Processing column: brand_name


100%|██████████| 1393287/1393287 [00:05<00:00, 272962.38it/s]


Finished processing brand_name
Processing column: item_description


...

Finished processing category_1
Processing column: category_2


100%|██████████| 1393287/1393287 [00:15<00:00, 88335.51it/s] 


Finished processing category_2
Processing column: category_3


100%|██████████| 1393287/1393287 [00:10<00:00, 127253.27it/s]


Finished processing category_3
All text preprocessing completed!


In [ ]:
# 데이터 전처리로 인해 새로 발생한 결측치 확인
train_df.isnull().sum()

In [ ]:
# 'name'과 'item_description'에서 새로 발생한 결측치 삭제
train_df = train_df.dropna(subset=['name', 'item_description'])

## 'brand_name' 결측치

### 텍스트 유추로 채우기

In [ ]:
# Create 'combined text' column by concatenating 'name' and 'item_description'
train_df['combined_text'] = train_df['name'] + ' ' + train_df['item_description']

# Strip leading/trailing spaces
train_df['combined_text'] = train_df['combined_text'].str.strip()

In [ ]:
brand_null_head_idx = train_df[train_df['brand_name'].isnull()].head(50).index.tolist()
train_df.iloc[brand_null_head_idx]

In [ ]:
brand_null_tail_idx = train_df[train_df['brand_name'].isnull()].tail(50).index.tolist()
train_df.iloc[brand_null_tail_idx]

In [ ]:
#  Count occurrences of each `brand_name` by `category_1`
brand_cat_counts_df = train_df.groupby(['brand_name', 'category_1']).size()
brand_cat_counts_df = pd.DataFrame(brand_cat_counts_df).reset_index()
brand_cat_counts_df.columns = ['brand_name', 'category_1', 'count']
brand_cat_counts_df = brand_cat_counts_df.sort_values(by='count', ascending=False)
brand_cat_counts_df

In [ ]:
# Filter for brands with counts > threshold
threshold = 10
brands_df = brand_cat_counts_df[brand_cat_counts_df['count'] > threshold]
brands_df

In [ ]:
# List of irrelevant brand names to remove
irrelevant_brands = ['all', 'm', 'ring']

# Remove rows where 'brand_name' is in the irrelevant brands list
brands_df = brands_df[~brands_df['brand_name'].isin(irrelevant_brands)]

In [ ]:
# Group by 'category_1' and aggregate 'brand_name' into lists
brand_cat_dict = brands_df.groupby('category_1')['brand_name'].apply(list).to_dict()

In [ ]:
# Sort the brand names in each category by length in descending order
for category in brand_cat_dict:
    brand_cat_dict[category] = sorted(brand_cat_dict[category], key=len, reverse=True)

# Function to find and update the brand based on `category_1`
def update_brand(row, brand_cat_dict):
    if pd.isnull(row['brand_name']) or row['brand_name'] == 'all':
        combined_text = row['combined_text']
        category = row['category_1']
        if category in brand_cat_dict:
            # Check for each brand in the category (longest first due to sorting)
            for brand in brand_cat_dict[category]:
                # Use word boundary to ensure exact match
                if re.search(r'\b' + re.escape(brand) + r'\b', combined_text):
                    return brand
    return row['brand_name']

chunk_size = 5000  # Adjust based on your memory capacity
num_chunks = (len(train_df) // chunk_size) + 1

# Initialize tqdm
tqdm.pandas()

# Process the data in chunks
processed_chunks = []
for i in tqdm(range(num_chunks)):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(train_df))
    chunk = train_df.iloc[start:end].copy()

    # Apply the update_brand function to each row in the chunk
    chunk['brand_name'] = chunk.progress_apply(lambda row: update_brand(row, brand_cat_dict), axis=1)

    processed_chunks.append(chunk)

    # Explicitly delete chunk and call garbage collection to free memory
    del chunk
    gc.collect()

# Concatenate all chunks back into a single DataFrame
train_df = pd.concat(processed_chunks).reset_index(drop=True)

print("Brand updating completed!")

### 'brand_name' 세부 조정

In [ ]:
# 'pink' > 'victorias secret' (if applicable)

# Define a function to check and update the brand name
def update_brand_pink(row):
    if row['brand_name'] == 'pink':
        if 'victorias secret' in row['combined_text'] or 'vs' in row['combined_text']:
            return 'victorias secret'
    return row['brand_name']

# Apply the function to each row in the dataframe using .loc to avoid SettingWithCopyWarning
train_df.loc[:, 'brand_name'] = train_df.apply(update_brand_pink, axis=1)

In [ ]:
# 'm' > 'motorola' (if applicable)

m_mask = train_df['brand_name'] == 'm'

for idx in tqdm(train_df[m_mask].index):
    if (('motorola' in train_df.loc[idx, 'name']) or
        ('motorola' in train_df.loc[idx, 'item_description'])):
        train_df.loc[idx, 'brand_name'] = 'motorola'

In [ ]:
# 'galaxy' to 'samsung' (if applicable)

galaxy_mask = train_df['brand_name'] == 'galaxy'

# Steps 2 and 3: For those rows, check 'name' and 'item_description', and update if necessary
for idx in tqdm(train_df[galaxy_mask].index):
    if (('samsung' in train_df.loc[idx, 'name']) or
        ('samsung' in train_df.loc[idx, 'item_description'])):
        train_df.loc[idx, 'brand_name'] = 'samsung'

In [ ]:
train_df.iloc[brand_null_head_idx]

In [ ]:
train_df.iloc[brand_null_tail_idx]

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['brand_name'].value_counts()

### 이외 결측치 'unknown'으로 일괄 대체

In [ ]:
# 결측치 처리

train_df['brand_name'] = train_df['brand_name'].fillna('unknown')

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['brand_name'].value_counts()

# Feature engineering

In [ ]:
# Feature engineering for 'item_description'

# List of phrases
phrases = [
    'brand new', 'never opened', 'with tag', 'new in box', 'great condition',
    'certificate of authenticity', 'complete set', 'worn once', 'great condition',
    'no stains', 'like new'
]

# Create dummy columns for each phrase
for phrase in phrases:
    column_name = phrase.replace(' ', '_')  # Replace spaces with underscores for column names
    train_df[column_name] = train_df['combined_text'].str.contains(phrase).astype(int)

# 불필요한 컬럼 삭제

In [ ]:
train_df.drop(['train_id', 'name', 'category_name', 'item_description', 'combined_text'], axis=1, inplace=True)

# 라벨인코딩

In [ ]:
# Label encoding for categorical columns

columns_to_encode = ['brand_name', 'category_1', 'category_2', 'category_3']

le = LabelEncoder()

for col in columns_to_encode:
    train_df[col] = le.fit_transform(train_df[col])

In [ ]:
train_df.head()

,item_condition_id,brand_name,price,shipping,category_1,category_2,category_3,brand_new,never_opened,with_tag,new_in_box,great_condition,certificate_of_authenticity,complete_set,worn_once,no_stains,like_new
0,3.0,3531,52.0,0.0,1,30,211,0,0,0,0,1,0,0,0,0,0
1,1.0,4149,10.0,1.0,9,102,94,0,0,0,0,0,0,0,0,0,0
2,1.0,4429,35.0,1.0,3,55,403,0,0,1,0,0,0,0,0,0,0
3,1.0,4429,44.0,0.0,9,58,534,0,0,0,0,0,1,0,0,0,0
4,3.0,418,59.0,0.0,9,72,557,0,0,0,0,0,0,0,0,0,0


# 전처리 완료된 버전 csv 파일로 저장

In [ ]:
# Define the save path
save_path = '/content/drive/MyDrive/Colab Notebooks/AIFFEL_DATATHONE(2조)/train_df_preprocessed_complete.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the DataFrame to a CSV file with proper encoding and quoting
train_df.to_csv(save_path, index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

print(f"File saved successfully to {save_path}")